In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data = pd.read_csv('../input/eeg-brainwave-dataset-feeling-emotions/emotions.csv')

In [ ]:
data

In [ ]:
sample = data.loc[0, 'fft_0_b':'fft_749_b']

plt.figure(figsize=(16, 10))
plt.plot(range(len(sample)), sample)
plt.title("Features fft_0_b through fft_749_b")
plt.show()

In [ ]:
data['label'].value_counts()

In [ ]:
label_mapping = {'NEGATIVE': 0, 'NEUTRAL': 1, 'POSITIVE': 2}

# Preprocessing

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    df['label'] = df['label'].replace(label_mapping)
    
    y = df['label'].copy()
    X = df.drop('label', axis=1).copy()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train

# Modeling

In [ ]:
def custom(x):
    return tf.keras.backend.square(tf.keras.backend.max(x)*tf.keras.backend.softmax(x))
    
ansdict = {"softmax":0,"custom":0}
model=0

for af in list(ansdict.keys()) :
    inputs = tf.keras.Input(shape=(X_train.shape[1],))
    expand_dims = tf.expand_dims(inputs, axis=2)
    gru = tf.keras.layers.GRU(256, return_sequences=True)(expand_dims)
    flatten = tf.keras.layers.Flatten()(gru)
    if af=="softmax":
        outputs = tf.keras.layers.Dense(3, activation=af)(flatten)
    else:
        outputs = tf.keras.layers.Dense(3, activation=custom)(flatten)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt=tf.keras.optimizers.SGD(lr=0.00001)
    model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )
    history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
    tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
    )
    ]
    )
    model_acc = model.evaluate(X_test, y_test, verbose=0)[1]
    ansdict[af]=model_acc
    print("Test Accuracy : {:.3f}%".format(model_acc * 100))

# Results

In [ ]:
ansdict=dict(sorted(ansdict.items(), key=lambda item: item[1]))
ansdict

In [ ]:
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(list(ansdict.keys()), list(ansdict.values()), 
        width = 0.1)
 
plt.xlabel("Activation Functions in increasing order of accuracy")
plt.ylabel("Accuracy in %")
plt.title("Accracy % for different activation functions")
plt.show()

In [ ]:
y_pred = np.array(list(map(lambda x: np.argmax(x), model.predict(X_test))))

cm = confusion_matrix(y_test, y_pred)
clr = classification_report(y_test, y_pred, target_names=label_mapping.keys())

plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
plt.xticks(np.arange(3) + 0.5, label_mapping.keys())
plt.yticks(np.arange(3) + 0.5, label_mapping.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n----------------------\n", clr)